In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

from nibabel.loadsave import ImageFileError

from nilearn.image import load_img
from nilearn import datasets
from nilearn.maskers import NiftiMasker, NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

from sklearn.preprocessing import StandardScaler

from nilearn import plotting

from itertools import combinations, product
from tqdm.notebook import tqdm

from scipy.stats import spearmanr, entropy

from scipy.ndimage import gaussian_filter1d, uniform_filter1d, maximum_filter1d, minimum_filter1d

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from scipy.linalg import logm

import pandas as pd

**Getting timeseries from fMRI scans**

In [2]:
atlas = datasets.fetch_atlas_aal()
atlas_filename = atlas.maps
atlas_labels = atlas.labels
masker = NiftiLabelsMasker(atlas_filename, detrend=False)

def get_time_series(path):
    # print(path)
    try:
        fmri_img = load_img(path)
    except ImageFileError:  # no run for session
        return None
    
    time_series = masker.fit_transform(fmri_img)
    time_series = StandardScaler().fit_transform(time_series[:, :])
    return time_series

In [3]:
def get_ts_by_info(subj_id, session_id, run_id):
    return get_time_series(f'./raw/timeseries/subj0{subj_id}/timeseries_session{session_id:02d}_run{run_id:02d}.nii.gz')

In [7]:
SUBJ_IDS = (1, 5)  # need range(1, 8+1) --- download
SESSION_IDS = range(1, 37+1)  # need range(1, 37+1) --- download, or use range(1, 23+1)
RUN_IDS = range(1, 14+1)  # need range(1, 14+1) --- check 13 and 14 - may be empty

trend_size = 50

T = []
X = []
y = []

for subj_id, session_id, run_id in tqdm(itertools.product(SUBJ_IDS, SESSION_IDS, RUN_IDS),
                                        total=len(SUBJ_IDS) * len(SESSION_IDS) * len(RUN_IDS),
                                        desc='Getting timeseries'):
    time_series = get_ts_by_info(subj_id, session_id, run_id)
    if time_series is None:
        continue
    
    # get trend and subtract it
    time_series_trend = uniform_filter1d(time_series, trend_size, axis=0)
    time_series_filtered = time_series - time_series_trend
                        
    # standardize
    time_series_std = StandardScaler().fit_transform(time_series)
    time_series_filtered_std = StandardScaler().fit_transform(time_series_filtered)
                        
    # get connectivity matrices
    A_raw = ConnectivityMeasure(kind="correlation").fit_transform([time_series_std])[0]
    A_filtered = ConnectivityMeasure(kind="correlation").fit_transform([time_series_filtered_std])[0]
    
    # set data
    T.append([time_series_std, time_series_filtered_std])
    X.append([A_raw, A_filtered])
    y.append(subj_id)
T = np.array(T)
X = np.array(X)
y = np.array(y)

Getting timeseries:   0%|          | 0/1036 [00:00<?, ?it/s]

In [32]:
n_trials = 478
n_steps = 226
n_rois = 116

# T = np.zeros((2 * n_trials, 2, n_steps, n_rois))
# X = np.zeros((2 * n_trials, 2, n_rois, n_rois)) # subject, raw/filtered, matrix
y_subjects = np.repeat([0, 1], n_trials)
y_sessions = np.tile(np.concatenate([np.repeat(0, 240), np.repeat(1, 238)]), 2)
y = np.concatenate([np.expand_dims(y_subjects, 0), np.expand_dims(y_sessions, 0)], axis=0)

In [33]:
np.save("T_subjects_1,5_nsd_vs_rest.npy", T)
np.save("X_subjects_1,5_nsd_vs_rest.npy", X)
np.save("y_subjects_1,5_nsd_vs_rest.npy", y)

In [68]:
T.shape, X.shape, y.shape

((956, 226, 116), (956, 66), (956,))

**Misc**

In [67]:
import os.path

subj_id, session_id, run_id = 1, 2, 13
print(os.path.getsize(f'./raw/timeseries/subj0{subj_id}/timeseries_session{session_id:02d}_run{run_id:02d}.nii.gz'))

359


In [131]:
z_idx = np.ix_(np.arange(0, X.shape[0]), [0, 1], [0, 1])
X_dmn[z_idx].shape, X_dmn[:, [0, 1], [0, 1]].shape

((956, 2, 2), (956, 2))

**Working with timeseries**

Loading data

In [4]:
T_raw = np.load("T_subjects_15_nsd_vs_rest.npy")[:, 1]
X_raw = np.load("X_subjects_15_nsd_vs_rest.npy")[:, 1]
y = np.load("y_subjects_15_nsd_vs_rest.npy")[1]
T_raw.shape, X_raw.shape, y.shape

((956, 226, 116), (956, 116, 116), (956,))

Brain networks

In [5]:
# visual cortex
visual = list(range(42, 53+1))
# sensorimotor network, somatomotor network (SMN)
smn = [0, 1, 6, 7, 18, 19, 56, 57, 62, 63, 68, 69]
# dorsal attention network (DAN), dorsal frontoparietal network (D-FPN)
dan = [28, 29, 30, 31, 84, 85]
# ventral attention network (VAN), ventral frontoparietal network (VFN), ventral attention system (VAS)
van = [32, 33, 34, 35, 36, 37, 52, 53, 62, 63, 64, 65]
# frontoparietal network (FPN), central executive network (CEN), lateral frontoparietal network (L-FPN)
fpn = [6, 7, 10, 11, 12, 13, 60, 61, 64, 65]
# limbic system, paleomammalian cortex
limbic = list(range(30, 39+1)) + list(range(80, 87+1))
# default mode network (DMN), default network, default state network, medial frontoparietal network (M-FPN)
dmn = [22, 23, 34, 35, 36, 37, 38, 39, 64, 65, 66, 67]

In [6]:
brain_networks = [visual, smn, dan, van, fpn, limbic, dmn]
brain_network_names = ["visual cortex (12 regions)",
                       "sensorimotor network (SMN) (12 regions)",
                       "dorsal attention network (DAN) (6 regions)",
                       "ventral attention network (VAN) (12 regions)",
                       "frontoparietal network (FPN) (10 regions)",
                       "limbic system (18 regions)",
                       "default mode network (DMN) (12 regions)"
                      ]
brain_short_network_names = ["visual (12)", "SMN (12)", "DAN (6)", "VAN (12)", "FPN (10)", "limbic (18)", "DMN (12)"]
brain_networks_with_names = list(zip(brain_networks, brain_network_names))
brain_networks_with_names

[([42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53],
  'visual cortex (12 regions)'),
 ([0, 1, 6, 7, 18, 19, 56, 57, 62, 63, 68, 69],
  'sensorimotor network (SMN) (12 regions)'),
 ([28, 29, 30, 31, 84, 85], 'dorsal attention network (DAN) (6 regions)'),
 ([32, 33, 34, 35, 36, 37, 52, 53, 62, 63, 64, 65],
  'ventral attention network (VAN) (12 regions)'),
 ([6, 7, 10, 11, 12, 13, 60, 61, 64, 65],
  'frontoparietal network (FPN) (10 regions)'),
 ([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 80, 81, 82, 83, 84, 85, 86, 87],
  'limbic system (18 regions)'),
 ([22, 23, 34, 35, 36, 37, 38, 39, 64, 65, 66, 67],
  'default mode network (DMN) (12 regions)')]

MI estimators

In [4]:
def bivariate_mutual_information(X, Y):
    r = np.corrcoef(X, Y)[0,1]
    return -0.5 * np.log(1 - r**2)

def H(sigma=1):
    return 0.5 * np.log(2 * np.pi * np.e * sigma)

def entropy(S):
    n = S.shape[0]
    return 0.5 * np.log((2 * np.pi * np.e) ** n * np.linalg.det(S))

def gaussI(S):
    return -0.5 * np.log(np.linalg.det(S))

In [5]:
def gaussU_dim3(S):
    if S.shape[0] != 3:
        return gaussI(S)
    rho_y_x2 = S[0, 2]
    rho_y_x1 = S[0, 1]
    if rho_y_x2 < rho_y_x1:
        return np.log((1 - rho_y_x2 ** 2) / (1 - rho_y_x1 ** 2)) / 2
    else:
        return 0

X_transformers

In [6]:
def corrmat2vec(A, take_logm=False):
    """
    Convert array of correlation matrices (A) to the object-feature table 
    
    A.shape = (k, n, n)
    k = number of runs
    n = number of timeseries
    A[i] = correlation matrix of i-th run
    
    log -- take matrix logarithm (standard symmetric positive definite matrix projection)
    """
    k, n = A.shape[0], A.shape[1]
    # note that A[i, j, j] = 1
    A_vec = np.zeros((k, n * (n - 1) // 2))
    triu_idx = np.triu_indices(n, 1)
    
    for i in range(k):
        cur = logm(A[i]) if take_logm else A[i]
        A_vec[i] = cur[triu_idx]
        
    return A_vec

In [25]:
def genXk(X_bn, k, MI_estimator):
    r = range(X_bn.shape[1])
    n = X_bn.shape[0]
    n_combs = len(list(combinations(r, k)))

    Xk = np.zeros((n, n_combs))

    for i, S in enumerate(X_bn):
        for j, simplex in enumerate(combinations(r, k)):
            simplex_idx = np.ix_(simplex, simplex)
            Xk[i, j] = MI_estimator(S[simplex_idx])
    return Xk

def down_genX3_gaussU_dim3(X_bn):
    k = 3
    r = range(X_bn.shape[1])
    n = X_bn.shape[0]
    n_combs = len(list(combinations(r, k)))

    Xk = np.zeros((n, n_combs))

    for i, S in enumerate(X_bn):
        for j, simplex in enumerate(combinations(r, k)):
            simplex = list(simplex)
            simplex[1], simplex[2] = simplex[2], simplex[1]
            simplex_idx = np.ix_(simplex, simplex)
            Xk[i, j] = gaussU_dim3(S[simplex_idx])
    return Xk

def get_gen_functor(ks, MI_estimator):
    return (lambda X_bn: np.hstack(tuple(genXk(X_bn, k, MI_estimator) for k in ks)))

def merge_functor(*args):
    return (lambda X_bn: np.hstack(tuple(f(X_bn) for f in args)))

In [20]:
# X_transformers = [corrmat2vec,
#                   get_gen_functor([2], mutual_information),
#                   get_gen_functor([3], mutual_information),
#                   get_gen_functor([4], mutual_information),
#                   get_gen_functor([5], mutual_information),
#                   get_gen_functor([6], mutual_information),
#                   get_gen_functor([2, 3], mutual_information),
#                   get_gen_functor([2, 3, 4, 5], mutual_information),
#                   get_gen_functor([2, 3, 4, 5, 6], mutual_information)
#                  ]
# X_transformer_names = ["correlation",
#                        "gaussian MI, k = 2",
#                        "gaussian MI, k = 3",
#                        "gaussian MI, k = 4",
#                        "gaussian MI, k = 5",
#                        "gaussian MI, k = 6",
#                        "gaussian MI, k = 2, 3",
#                        "gaussian MI, k = 2, 3, 4, 5",
#                        "gaussian MI, k = 2, 3, 4, 5, 6",
#                       ]
# X_transformer_with_names = list(zip(X_transformers, X_transformer_names))
X_transformer_with_names = [(corrmat2vec, "correlation"),
                            (get_gen_functor([2], gaussI), "gaussian MI, k = 2"),
                            (get_gen_functor([3], gaussI), "gaussian MI, k = 3"),
                            (get_gen_functor([2, 3], gaussI), "gaussian MI, k = 2, 3"),
                            (get_gen_functor([3], gaussU_dim3), "gaussI + gaussU on 3 dim, k = 3"),
                            (get_gen_functor([2, 3], gaussU_dim3), "gaussI + gaussU on 3 dim, k = 2, 3"),
                            (get_gen_functor([4], gaussI), "gaussian MI, k = 4"),
                            (get_gen_functor([2, 3, 4], gaussI), "gaussian MI, k = 2, 3, 4"),
                            (get_gen_functor([2, 3, 4], gaussU_dim3), "gaussI + gaussU on 3 dim, k = 2, 3, 4"),
                           ]

Scorers

In [7]:
def logistic_score(X, y, n_repeats=10, n_splits=5):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31, shuffle=True)
#     scaler = StandardScaler()
#     model = LogisticRegression(C=5, max_iter=500, random_state=31)
#     clf = Pipeline([("scaler", scaler), ("model", model)]).fit(X_train, y_train)
    accuracy = np.zeros((n_repeats, n_splits))
    for i in tqdm(range(n_repeats)):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=i)
        for j, (train_idx, test_idx) in enumerate(skf.split(X, y)):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
    
            scaler = StandardScaler()
            model = LogisticRegression(C=5, max_iter=1500, random_state=i*n_repeats+j)
            clf = Pipeline([("scaler", scaler), ("model", model)]).fit(X_train, y_train)

            y_hat = clf.predict(X_test)

            accuracy[i, j] = accuracy_score(y_test, y_hat)
    # convenient in jupyter notebook: instantly print accuracy and do not print the last line return value (which could be an accuracy table)
    return f"{accuracy.mean():.5f} ± {accuracy.std():.5f}"

In [28]:
def mlp_score(X, y, n_repeats=10, n_splits=5):
    accuracy = np.zeros((n_repeats, n_splits))
    for i in tqdm(range(n_repeats)):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=i)
        for j, (train_idx, test_idx) in enumerate(skf.split(X, y)):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            scaler = StandardScaler()
            model = MLPClassifier(random_state=i*n_repeats+j)
            clf = Pipeline([("scaler", scaler), ("model", model)]).fit(X_train, y_train)

            y_hat = clf.predict(X_test)

            accuracy[i, j] = accuracy_score(y_test, y_hat)
    # convenient in jupyter notebook: instantly print accuracy and do not print the last line return value (which could be an accuracy table)
    return f"{accuracy.mean():.5f} ± {accuracy.std():.5f}"

In [45]:
def catboost_score(X, y, n_repeats=10, n_splits=5):
    accuracy = np.zeros((n_repeats, n_splits))
    for i in tqdm(range(n_repeats)):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=i)
        for j, (train_idx, test_idx) in enumerate(skf.split(X, y)):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            clf = CatBoostClassifier(random_seed=i*n_repeats+j, verbose=False).fit(X_train, y_train)

            y_hat = clf.predict(X_test)

            accuracy[i, j] = accuracy_score(y_test, y_hat)
    # convenient in jupyter notebook: instantly print accuracy and do not print the last line return value (which could be an accuracy table)
    return f"{accuracy.mean():.5f} ± {accuracy.std():.5f}"

In [30]:
scorers = [logistic_score, mlp_score, catboost_score]
scorer_names = ["Logistic", "MLP", "CatBoost"]
scorers_with_names = list(zip(scorers, scorer_names))

Full test

In [8]:
def cor_test_on_brain_networks(X_raw, y, brain_networks_with_names, X_transformer, scorer, n_repeats=10, ignore=tuple()):
    """
    Test, how well feature generator X_transformer works on each brain network
    TODO: X_transformer will get T_raw, now it get X_raw -- T_raw's correlation matrix
    NOTE: scorer is simple crossvalidation with logistic regression;
    for testing heavy simplicial models X_transformer(T_raw) = encoder(diagram(simplicial complex(T_raw))),
    simplicial complex can be not full (particular dims, subsampling, <eps pruning),
    scorer splits data in train/val (for hyperparameters)/test and uses decoder classifier
    """
    scores = [None] * len(brain_networks_with_names)
    for i, (brain_network, brain_network_name) in enumerate(brain_networks_with_names):
        if i in ignore:
            print(f"{brain_network_name} ignored")
            scores[i] = "Too long"
            continue
        bn_idx = np.ix_(np.arange(0, X_raw.shape[0]), brain_network, brain_network)
        X_bn = X_raw[bn_idx]
        X = X_transformer(X_bn)
        score = scorer(X, y, n_repeats=n_repeats)
        # print(f"Score {score} on {brain_network_name} network with {len(brain_network)} regions")
        scores[i] = score
    return np.array(scores)

In [9]:
def ts_test_on_brain_networks(T_raw, y, brain_networks_with_names, T_transformer, scorer, n_repeats=10, ignore=tuple()):
    """
    Test, how well feature generator X_transformer works on each brain network
    TODO: X_transformer will get T_raw, now it get X_raw -- T_raw's correlation matrix
    NOTE: scorer is simple crossvalidation with logistic regression;
    for testing heavy simplicial models X_transformer(T_raw) = encoder(diagram(simplicial complex(T_raw))),
    simplicial complex can be not full (particular dims, subsampling, <eps pruning),
    scorer splits data in train/val (for hyperparameters)/test and uses decoder classifier
    """
    scores = [None] * len(brain_networks_with_names)
    for i, (brain_network, brain_network_name) in enumerate(brain_networks_with_names):
        if i in ignore:
            print(f"{brain_network_name} ignored")
            scores[i] = "Too long"
            continue
        T_bn = T_raw[:, :, brain_network]
        X = T_transformer(T_bn)
        score = scorer(X, y, n_repeats=n_repeats)
        # print(f"Score {score} on {brain_network_name} network with {len(brain_network)} regions")
        scores[i] = score
    return np.array(scores)

In [10]:
def full_test(T_raw, X_raw, y,
              brain_networks_with_names,
              X_transformer_with_names,
              T_transformer_with_names,
              scorers_with_names,
              n_repeats=10):
    _, brain_network_names = zip(*brain_networks_with_names)
    X_transformer_names = []
    T_transformer_names = []
    if X_transformer_with_names is not None:
        X_transformers, X_transformer_names = zip(*X_transformer_with_names)
    if T_transformer_with_names is not None:
        T_transformers, T_transformer_names = zip(*T_transformer_with_names)
    ret = []
    for scorer, scorer_name in scorers_with_names:
        res = []
        if X_transformer_with_names is not None:
            for X_transformer, X_transformer_name in X_transformer_with_names:
                cur = cor_test_on_brain_networks(X_raw, y, brain_networks_with_names, X_transformer, scorer, n_repeats=n_repeats)
                res.append(cur)
                print(f"Scores {cur} by scorer {scorer_name} with {X_transformer_name} transformer")
        if T_transformer_with_names is not None:
            for T_transformer, T_transformer_name in T_transformer_with_names:
                cur = ts_test_on_brain_networks(T_raw, y, brain_networks_with_names, T_transformer, scorer, n_repeats=n_repeats)
                res.append(cur)
                print(f"Scores {cur} by scorer {scorer_name} with {T_transformer_name} transformer")
        df = pd.DataFrame(data=res, columns=brain_network_names, index=X_transformer_names + T_transformer_names)
        df.style.set_caption(f"Scorer: {scorer_name}")
        ret.append(df)
    return ret

In [26]:
brain_networks = [visual, smn, dan, van, fpn, dmn]
brain_network_names = ["visual cortex (12 regions)",
                       "sensorimotor network (SMN) (12 regions)",
                       "dorsal attention network (DAN) (6 regions)",
                       "ventral attention network (VAN) (12 regions)",
                       "frontoparietal network (FPN) (10 regions)",
                       "default mode network (DMN) (12 regions)"
                      ]
brain_networks_with_names = list(zip(brain_networks, brain_network_names))

X_transformer_with_names = [(corrmat2vec, "correlation"),
                            (get_gen_functor([2], gaussI), "gaussI, k = 2"),
                            (get_gen_functor([3], gaussI), "gaussI, k = 3"),
                            (get_gen_functor([2, 3], gaussI), "gaussI, k = 2, 3"),
                            (get_gen_functor([3], gaussU_dim3), "up gaussU, k = 3"),
                            (down_genX3_gaussU_dim3, "down gaussU, k = 3"),
                            (merge_functor(get_gen_functor([3], gaussU_dim3), down_genX3_gaussU_dim3), "both gaussU, k = 3"),
                            (merge_functor(get_gen_functor([3], gaussI),
                                           get_gen_functor([3], gaussU_dim3),
                                           down_genX3_gaussU_dim3), "gaussI + both gaussU, k = 3"),
                            (merge_functor(get_gen_functor([2, 3], gaussI),
                                           get_gen_functor([3], gaussU_dim3),
                                           down_genX3_gaussU_dim3), "gaussI (2, 3) + both gaussU (3)")
                           ]

scorers = [logistic_score]
scorer_names = ["Logistic"]
scorers_with_names = list(zip(scorers, scorer_names))

ret = full_test(X_raw, y, brain_networks_with_names, X_transformer_with_names, scorers_with_names, n_repeats=10)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Scores ['0.76267 ± 0.02707' '0.73744 ± 0.02576' '0.70220 ± 0.02781'
 '0.75334 ± 0.02811' '0.75284 ± 0.03669' '0.74718 ± 0.02697'] by scorer Logistic with gaussI (2, 3) + both gaussU (3) transformer


In [27]:
ret[0]

,visual cortex (12 regions),sensorimotor network (SMN) (12 regions),dorsal attention network (DAN) (6 regions),ventral attention network (VAN) (12 regions),frontoparietal network (FPN) (10 regions),default mode network (DMN) (12 regions)
correlation,0.78515 ± 0.02589,0.76831 ± 0.02676,0.67606 ± 0.03043,0.77134 ± 0.02904,0.76340 ± 0.02740,0.78139 ± 0.02631
"gaussI, k = 2",0.76342 ± 0.02634,0.76234 ± 0.02594,0.66495 ± 0.02922,0.78514 ± 0.02513,0.77124 ± 0.02648,0.77877 ± 0.02603
"gaussI, k = 3",0.78737 ± 0.02701,0.77238 ± 0.02184,0.68911 ± 0.02680,0.80387 ± 0.02596,0.77187 ± 0.02646,0.79048 ± 0.02947
"gaussI, k = 2, 3",0.78214 ± 0.02561,0.78421 ± 0.02253,0.70240 ± 0.03299,0.79508 ± 0.02557,0.77009 ± 0.02804,0.79645 ± 0.02933
"up gaussU, k = 3",0.77742 ± 0.02769,0.73984 ± 0.02952,0.63327 ± 0.02481,0.75126 ± 0.02988,0.76236 ± 0.03006,0.74362 ± 0.02823
"down gaussU, k = 3",0.74342 ± 0.02984,0.71819 ± 0.03062,0.60973 ± 0.03156,0.71821 ± 0.02724,0.74195 ± 0.03248,0.73462 ± 0.03074
"both gaussU, k = 3",0.75734 ± 0.02968,0.72384 ± 0.02765,0.64341 ± 0.02683,0.73577 ± 0.02614,0.74970 ± 0.02810,0.73315 ± 0.02655
"gaussI + both gaussU, k = 3",0.76256 ± 0.02757,0.73545 ± 0.02607,0.69529 ± 0.02607,0.75103 ± 0.02854,0.75304 ± 0.03558,0.74791 ± 0.02686
"gaussI (2, 3) + both gaussU (3)",0.76267 ± 0.02707,0.73744 ± 0.02576,0.70220 ± 0.02781,0.75334 ± 0.02811,0.75284 ± 0.03669,0.74718 ± 0.02697


In [28]:
np.save('experiment_table_gaussU.npy', ret[0].to_numpy())

In [11]:
def full_experiment(X_raw, y, scorer):
    res = []
    TOO_LONG = {brain_short_network_names.index("limbic")}
    for i, X_transformer in enumerate(X_transformers):
        cur = test_on_brain_networks(X_raw, y, X_transformer, scorer,
                                     ignore=TOO_LONG if X_transformer_names[i].count("6") else set())
        print(cur)
        res.append(cur)
    return pd.DataFrame(data=res, columns=brain_network_names, index=X_transformer_names)

In [24]:
experiment_table = full_experiment(X_raw, y, logistic_score)

  0%|          | 0/10 [00:00<?, ?it/s]

Score 0.78515 ± 0.02589 on visual cortex network with 12 regions


  0%|          | 0/10 [00:00<?, ?it/s]

Score 0.76831 ± 0.02676 on sensorimotor network (SMN) network with 12 regions


  0%|          | 0/10 [00:00<?, ?it/s]

...

Score 0.78450 ± 0.02787 on ventral attention network (VAN) network with 12 regions


  0%|          | 0/10 [00:00<?, ?it/s]

Score 0.77815 ± 0.02902 on frontoparietal network (FPN) network with 10 regions
limbic system ignored


  0%|          | 0/10 [00:00<?, ?it/s]

Score 0.79267 ± 0.02504 on default mode network (DMN) network with 12 regions
['0.79029 ± 0.02631' '0.79267 ± 0.02654' '0.71067 ± 0.03199'
 '0.78450 ± 0.02787' '0.77815 ± 0.02902' 'Too long' '0.79267 ± 0.02504']


In [25]:
experiment_table

,visual cortex,sensorimotor network (SMN),dorsal attention network (DAN),ventral attention network (VAN),frontoparietal network (FPN),limbic system,default mode network (DMN)
correlation,0.78515 ± 0.02589,0.76831 ± 0.02676,0.67606 ± 0.03043,0.77134 ± 0.02904,0.76340 ± 0.02740,0.81234 ± 0.03431,0.78139 ± 0.02631
"gaussian MI, k = 2",0.76342 ± 0.02634,0.76234 ± 0.02594,0.66495 ± 0.02922,0.78514 ± 0.02513,0.77124 ± 0.02648,0.79372 ± 0.02850,0.77877 ± 0.02603
"gaussian MI, k = 3",0.78737 ± 0.02701,0.77238 ± 0.02184,0.68911 ± 0.02680,0.80387 ± 0.02596,0.77187 ± 0.02646,0.81150 ± 0.02835,0.79048 ± 0.02947
"gaussian MI, k = 4",0.79238 ± 0.02705,0.79226 ± 0.02512,0.67353 ± 0.02824,0.79874 ± 0.02520,0.76874 ± 0.02678,0.81976 ± 0.02637,0.79938 ± 0.03009
"gaussian MI, k = 5",0.78882 ± 0.02387,0.79822 ± 0.02795,0.59382 ± 0.03170,0.79843 ± 0.02492,0.76382 ± 0.02869,0.82865 ± 0.02632,0.80021 ± 0.03028
"gaussian MI, k = 6",0.78569 ± 0.02312,0.79926 ± 0.02676,0.58462 ± 0.02989,0.79530 ± 0.02474,0.76675 ± 0.02809,Too long,0.79707 ± 0.02925
"gaussian MI, k = 2, 3",0.78214 ± 0.02561,0.78421 ± 0.02253,0.70240 ± 0.03299,0.79508 ± 0.02557,0.77009 ± 0.02804,0.80973 ± 0.03131,0.79645 ± 0.02933
"gaussian MI, k = 2, 3, 4, 5",0.79175 ± 0.02669,0.79770 ± 0.02432,0.71078 ± 0.03205,0.78806 ± 0.02731,0.77669 ± 0.02942,0.81893 ± 0.02575,0.79686 ± 0.02352
"gaussian MI, k = 2, 3, 4, 5, 6",0.79029 ± 0.02631,0.79267 ± 0.02654,0.71067 ± 0.03199,0.78450 ± 0.02787,0.77815 ± 0.02902,Too long,0.79267 ± 0.02504


In [48]:
experiment_table

NameError: name 'experiment_table' is not defined

In [43]:
np.save('experiment_table_data.npy', experiment_table.to_numpy())

In [45]:
pd.DataFrame(data=np.load('experiment_table_data.npy', allow_pickle=True),
             columns=brain_network_names, index=X_transformer_names)

,visual cortex,sensorimotor network (SMN),dorsal attention network (DAN),ventral attention network (VAN),frontoparietal network (FPN),limbic system,default mode network (DMN)
correlation,0.78515 ± 0.02589,0.76831 ± 0.02676,0.67606 ± 0.03043,0.77134 ± 0.02904,0.76340 ± 0.02740,0.81234 ± 0.03431,0.78139 ± 0.02631
"gaussian MI, k = 2",0.76342 ± 0.02634,0.76234 ± 0.02594,0.66495 ± 0.02922,0.78514 ± 0.02513,0.77124 ± 0.02648,0.79372 ± 0.02850,0.77877 ± 0.02603
"gaussian MI, k = 3",0.78737 ± 0.02701,0.77238 ± 0.02184,0.68911 ± 0.02680,0.80387 ± 0.02596,0.77187 ± 0.02646,0.81150 ± 0.02835,0.79048 ± 0.02947
"gaussian MI, k = 4",0.79238 ± 0.02705,0.79226 ± 0.02512,0.67353 ± 0.02824,0.79874 ± 0.02520,0.76874 ± 0.02678,0.81976 ± 0.02637,0.79938 ± 0.03009
"gaussian MI, k = 5",0.78882 ± 0.02387,0.79822 ± 0.02795,0.59382 ± 0.03170,0.79843 ± 0.02492,0.76382 ± 0.02869,0.82865 ± 0.02632,0.80021 ± 0.03028
"gaussian MI, k = 6",0.78569 ± 0.02312,0.79926 ± 0.02676,0.58462 ± 0.02989,0.79530 ± 0.02474,0.76675 ± 0.02809,Too long,0.79707 ± 0.02925
"gaussian MI, k = 2, 3",0.78214 ± 0.02561,0.78421 ± 0.02253,0.70240 ± 0.03299,0.79508 ± 0.02557,0.77009 ± 0.02804,0.80973 ± 0.03131,0.79645 ± 0.02933
"gaussian MI, k = 2, 3, 4, 5",0.79175 ± 0.02669,0.79770 ± 0.02432,0.71078 ± 0.03205,0.78806 ± 0.02731,0.77669 ± 0.02942,0.81893 ± 0.02575,0.79686 ± 0.02352
"gaussian MI, k = 2, 3, 4, 5, 6",0.79029 ± 0.02631,0.79267 ± 0.02654,0.71067 ± 0.03199,0.78450 ± 0.02787,0.77815 ± 0.02902,Too long,0.79267 ± 0.02504


In [65]:
experiment_table = pd.DataFrame(data=np.load('experiment_table_data.npy', allow_pickle=True),
                                 columns=["visual cortex (12 regions)",
                       "sensorimotor network (SMN) (12 regions)",
                       "dorsal attention network (DAN) (6 regions)",
                       "ventral attention network (VAN) (12 regions)",
                       "frontoparietal network (FPN) (10 regions)",
                       "limbic system (18 regions)",
                       "default mode network (DMN) (12 regions)"
                      ], index=tmp.iloc[:, 0].to_numpy())

In [66]:
experiment_table

,visual cortex (12 regions),sensorimotor network (SMN) (12 regions),dorsal attention network (DAN) (6 regions),ventral attention network (VAN) (12 regions),frontoparietal network (FPN) (10 regions),limbic system (18 regions),default mode network (DMN) (12 regions)
correlation,0.78515 ± 0.02589,0.76831 ± 0.02676,0.67606 ± 0.03043,0.77134 ± 0.02904,0.76340 ± 0.02740,0.81234 ± 0.03431,0.78139 ± 0.02631
"gaussian MI, k = 2",0.76342 ± 0.02634,0.76234 ± 0.02594,0.66495 ± 0.02922,0.78514 ± 0.02513,0.77124 ± 0.02648,0.79372 ± 0.02850,0.77877 ± 0.02603
"gaussian MI, k = 3",0.78737 ± 0.02701,0.77238 ± 0.02184,0.68911 ± 0.02680,0.80387 ± 0.02596,0.77187 ± 0.02646,0.81150 ± 0.02835,0.79048 ± 0.02947
"gaussian MI, k = 4",0.79238 ± 0.02705,0.79226 ± 0.02512,0.67353 ± 0.02824,0.79874 ± 0.02520,0.76874 ± 0.02678,0.81976 ± 0.02637,0.79938 ± 0.03009
"gaussian MI, k = 5",0.78882 ± 0.02387,0.79822 ± 0.02795,0.59382 ± 0.03170,0.79843 ± 0.02492,0.76382 ± 0.02869,0.82865 ± 0.02632,0.80021 ± 0.03028
"gaussian MI, k = 6",0.78569 ± 0.02312,0.79926 ± 0.02676,0.58462 ± 0.02989,0.79530 ± 0.02474,0.76675 ± 0.02809,Too long,0.79707 ± 0.02925
"gaussian MI, k = 2, 3",0.78214 ± 0.02561,0.78421 ± 0.02253,0.70240 ± 0.03299,0.79508 ± 0.02557,0.77009 ± 0.02804,0.80973 ± 0.03131,0.79645 ± 0.02933
"gaussian MI, k = 2, 3, 4, 5",0.79175 ± 0.02669,0.79770 ± 0.02432,0.71078 ± 0.03205,0.78806 ± 0.02731,0.77669 ± 0.02942,0.81893 ± 0.02575,0.79686 ± 0.02352
"gaussian MI, k = 2, 3, 4, 5, 6",0.79029 ± 0.02631,0.79267 ± 0.02654,0.71067 ± 0.03199,0.78450 ± 0.02787,0.77815 ± 0.02902,Too long,0.79267 ± 0.02504


In [82]:
experiment_table.to_csv('experiment_table.csv', sep=';')

In [63]:
tmp = pd.read_csv('experiment_table.csv', delimiter=',')

In [64]:
tmp

,Unnamed: 0,visual cortex,sensorimotor network (SMN),dorsal attention network (DAN),ventral attention network (VAN),frontoparietal network (FPN),limbic system,default mode network (DMN)
0,correlation,0.78515 ± 0.02589,0.76831 ± 0.02676,0.67606 ± 0.03043,0.77134 ± 0.02904,0.76340 ± 0.02740,0.81234 ± 0.03431,0.78139 ± 0.02631
1,"gaussian MI, k = 2",0.76342 ± 0.02634,0.76234 ± 0.02594,0.66495 ± 0.02922,0.78514 ± 0.02513,0.77124 ± 0.02648,0.79372 ± 0.02850,0.77877 ± 0.02603
2,"gaussian MI, k = 3",0.78737 ± 0.02701,0.77238 ± 0.02184,0.68911 ± 0.02680,0.80387 ± 0.02596,0.77187 ± 0.02646,0.81150 ± 0.02835,0.79048 ± 0.02947
3,"gaussian MI, k = 4",0.79238 ± 0.02705,0.79226 ± 0.02512,0.67353 ± 0.02824,0.79874 ± 0.02520,0.76874 ± 0.02678,0.81976 ± 0.02637,0.79938 ± 0.03009
4,"gaussian MI, k = 5",0.78882 ± 0.02387,0.79822 ± 0.02795,0.59382 ± 0.03170,0.79843 ± 0.02492,0.76382 ± 0.02869,0.82865 ± 0.02632,0.80021 ± 0.03028
5,"gaussian MI, k = 6",0.78569 ± 0.02312,0.79926 ± 0.02676,0.58462 ± 0.02989,0.79530 ± 0.02474,0.76675 ± 0.02809,Too long,0.79707 ± 0.02925
6,"gaussian MI, k = 2, 3",0.78214 ± 0.02561,0.78421 ± 0.02253,0.70240 ± 0.03299,0.79508 ± 0.02557,0.77009 ± 0.02804,0.80973 ± 0.03131,0.79645 ± 0.02933
7,"gaussian MI, k = 2, 3, 4, 5",0.79175 ± 0.02669,0.79770 ± 0.02432,0.71078 ± 0.03205,0.78806 ± 0.02731,0.77669 ± 0.02942,0.81893 ± 0.02575,0.79686 ± 0.02352
8,"gaussian MI, k = 2, 3, 4, 5, 6",0.79029 ± 0.02631,0.79267 ± 0.02654,0.71067 ± 0.03199,0.78450 ± 0.02787,0.77815 ± 0.02902,Too long,0.79267 ± 0.02504


In [55]:
tmp.iloc[:, 0].to_numpy()

array(['correlation', 'gaussian MI, k = 2', 'gaussian MI, k = 3',
       'gaussian MI, k = 4', 'gaussian MI, k = 5', 'gaussian MI, k = 6',
       'gaussian MI, k = 2, 3', 'gaussian MI, k = 2, 3, 4, 5',
       'gaussian MI, k = 2, 3, 4, 5, 6'], dtype=object)

In [59]:
tmp.columns.to_numpy()

array(['Unnamed: 0', 'visual cortex', 'sensorimotor network (SMN)',
       'dorsal attention network (DAN)',
       'ventral attention network (VAN)', 'frontoparietal network (FPN)',
       'limbic system', 'default mode network (DMN)'], dtype=object)

**T_transformer**

In [12]:
from scipy.special import gamma, digamma
from sklearn.neighbors import NearestNeighbors

def log_safe(vec):
    vec[vec<=0] = 1
    return np.log(vec)

def vol(d):
    numerator = np.pi ** (d/2)
    denominator = gamma((d/2) + 1)
    return numerator / denominator

def H_kl_delattre(X, n_neighbors=1):
    n, d = X.shape
    distances, _ = NearestNeighbors(n_neighbors=n_neighbors+1).fit(X).kneighbors(X)
    distances_k = distances[:,-1]
    Y = n * distances_k ** d
    
    return np.mean(log_safe(Y)) - digamma(n_neighbors) + np.log(vol(d))

def I(P, n_neighbors=1):
    n, d = P.shape
    
    H_P = H_kl_delattre(P, n_neighbors)
    
    H_marginal = np.zeros(d)
    for i, idx in enumerate(combinations(range(d), 1)):
        P_marginal_i = P[:,idx]
        H_marginal[i] = H_kl_delattre(P_marginal_i, n_neighbors)
        
    return np.sum(H_marginal) - H_P

In [13]:
def genTk(T_bn, k, MI_estimator):
    r = range(T_bn.shape[2])
    n = T_bn.shape[0]
    n_combs = len(list(combinations(r, k)))

    Tk = np.zeros((n, n_combs))

    for i, ts in enumerate(T_bn):
        for j, simplex in enumerate(combinations(r, k)):
            Tk[i, j] = MI_estimator(ts[:, simplex])
    return Tk

def Tget_gen_functor(ks, MI_estimator):
    return (lambda T_bn: np.hstack(tuple(genTk(T_bn, k, MI_estimator) for k in ks)))

In [14]:
brain_networks = [visual, smn, dan, van, fpn, dmn]
brain_network_names = ["visual cortex (12 regions)",
                       "sensorimotor network (SMN) (12 regions)",
                       "dorsal attention network (DAN) (6 regions)",
                       "ventral attention network (VAN) (12 regions)",
                       "frontoparietal network (FPN) (10 regions)",
                       "default mode network (DMN) (12 regions)"
                      ]
brain_networks_with_names = list(zip(brain_networks, brain_network_names))

T_transformer_with_names = [(Tget_gen_functor([2], I), "T2"),
                            (Tget_gen_functor([3], I), "T3"),
                            (Tget_gen_functor([2, 3], I), "T2, 3")
                           ]

scorers = [logistic_score]
scorer_names = ["Logistic"]
scorers_with_names = list(zip(scorers, scorer_names))

ret = full_test(T_raw, None, y,
                brain_networks_with_names,
                None,
                T_transformer_with_names,
                scorers_with_names, n_repeats=10)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Scores ['0.66297 ± 0.03182' '0.65020 ± 0.03249' '0.58890 ± 0.03377'
 '0.68680 ± 0.03204' '0.66172 ± 0.02950' '0.68661 ± 0.03229'] by scorer Logistic with T2 transformer


KeyboardInterrupt: 

In [59]:
from scipy.stats import spearmanr, entropy

from numba import int32, double    # import the types
from numba.experimental import jitclass

spec = [
    ('n', int32),
    ('d', int32),
    ('C', double[:, :]),
    ('P', double[:, :])
]

@jitclass(spec)
class Copula:    
    def __init__(self, XY, XY_order):
        
        self.n, self.d = XY.shape
#         if self.d<2:
#             raise ValueError(f"Data should be at least two-dimensional, but d={self.d}.")
            
        # order statistics
        ## XY_order = np.sort(XY, axis=0)
        
        # set C
        self.C = np.zeros((self.n,self.n))

        for i in range(self.n):
            for j in range(self.n):
                num = np.sum(np.logical_and(XY[:,0] <= XY_order[i,0], XY[:,1] <= XY_order[j,1])) / self.n
                self.C[i,j] = num

        # set P
        i_n = np.arange(1, self.n+1) / self.n
        j_n = np.arange(1, self.n+1) / self.n
        self.P = np.outer(i_n, j_n)
        
    def spearman(self):

#         if self.d!=2:
#             raise ValueError(f"Spearman's rho is the bivariate measure, but d={self.d}.")

        h_rho = 12/(self.n**2-1)
        return h_rho * np.sum(self.C - self.P) # TODO: general d

    def schweizer_wolff(self, norm="inf"):

#         if norm not in [1, 2, "inf"]:
#             raise ValueError("Norm should be '1', '2', or 'inf'.")

        # normalizing factors
        h_1 = ((1/(self.d+1)) - (1/(2**self.d))) ** -1
        h_2 = 90 # TODO
        h_inf = ((1/self.d)**(1/(self.d-1)) * (1 - (1/self.d)))**-1

        if norm==1:
            delta = h_1 / (self.n**self.d-1) * np.sum(np.abs(self.C - self.P))
        elif norm==2:
            delta_sq = h_2 / (self.n**self.d-1) * np.sum((self.C - self.P)**2)
            delta = np.sqrt(delta_sq)
        elif norm=="inf":
            delta = h_inf * np.max(np.abs(self.C - self.P))

        return delta

In [41]:
def ts_test_on_brain_networks(T_raw, y, brain_networks_with_names, T_transformer, scorer, n_repeats=10, ignore=tuple()):
    """
    Test, how well feature generator X_transformer works on each brain network
    TODO: X_transformer will get T_raw, now it get X_raw -- T_raw's correlation matrix
    NOTE: scorer is simple crossvalidation with logistic regression;
    for testing heavy simplicial models X_transformer(T_raw) = encoder(diagram(simplicial complex(T_raw))),
    simplicial complex can be not full (particular dims, subsampling, <eps pruning),
    scorer splits data in train/val (for hyperparameters)/test and uses decoder classifier
    """
    scores = [None] * len(brain_networks_with_names)
    for i, (brain_network, brain_network_name) in enumerate(brain_networks_with_names):
        if i in ignore:
            print(f"{brain_network_name} ignored")
            scores[i] = "Too long"
            continue
        T_bn = T_raw[:, :, brain_network]
        X = T_transformer(T_bn)
        score = scorer(X, y, n_repeats=n_repeats)
        # print(f"Score {score} on {brain_network_name} network with {len(brain_network)} regions")
        scores[i] = score
    return np.array(scores)

In [45]:
brain_networks = [visual, smn, dan, van, fpn, dmn]
brain_network_names = ["visual cortex (12 regions)",
                       "sensorimotor network (SMN) (12 regions)",
                       "dorsal attention network (DAN) (6 regions)",
                       "ventral attention network (VAN) (12 regions)",
                       "frontoparietal network (FPN) (10 regions)",
                       "default mode network (DMN) (12 regions)"
                      ]
brain_networks_with_names = list(zip(brain_networks, brain_network_names))
copulas_res = []
for brain_network, brain_network_name in brain_networks_with_names:
    T_bn = T_raw[:, :, brain_network]

    k = 2
    r = range(T_bn.shape[2])
    n = T_bn.shape[0]
    n_combs = len(list(combinations(r, k)))

    Tk_spearman2 = np.zeros((n, n_combs))
    Tk_schweizer_wolff2 = np.zeros((n, n_combs))

    for i, ts in tqdm(enumerate(T_bn), total=len(T_bn)):
        for j, simplex in enumerate(combinations(r, k)):
            cur = Copula(ts[:, simplex], np.sort(ts[:, simplex], axis=0))
            Tk_spearman2[i, j] = cur.spearman()
            Tk_schweizer_wolff2[i, j] = cur.schweizer_wolff()
    
    k = 3
    n_combs = len(list(combinations(r, k)))

    Tk_spearman3 = np.zeros((n, n_combs))
    Tk_schweizer_wolff3 = np.zeros((n, n_combs))

    for i, ts in tqdm(enumerate(T_bn), total=len(T_bn)):
        for j, simplex in enumerate(combinations(r, k)):
            cur = Copula(ts[:, simplex], np.sort(ts[:, simplex], axis=0))
            Tk_spearman3[i, j] = cur.spearman()
            Tk_schweizer_wolff3[i, j] = cur.schweizer_wolff()
    
    copulas_res.append([logistic_score(Tk_spearman2, y), logistic_score(Tk_spearman3, y),
                        logistic_score(np.hstack((Tk_spearman2, Tk_spearman3)), y),
                        logistic_score(Tk_schweizer_wolff2, y), logistic_score(Tk_schweizer_wolff3, y),
                        logistic_score(np.hstack((Tk_schweizer_wolff2, Tk_schweizer_wolff3)), y)
                       ])
    print(copulas_res[-1], brain_network_name)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

['0.77460 ± 0.02941', '0.76163 ± 0.02914', '0.77334 ± 0.03067', '0.74415 ± 0.02743', '0.73934 ± 0.02596', '0.74321 ± 0.02738'] sensorimotor network (SMN) (12 regions)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [70]:
brain_networks = [visual, smn, dan, van, fpn, dmn]
brain_network_names = ["visual cortex (12 regions)",
                       "sensorimotor network (SMN) (12 regions)",
                       "dorsal attention network (DAN) (6 regions)",
                       "ventral attention network (VAN) (12 regions)",
                       "frontoparietal network (FPN) (10 regions)",
                       "default mode network (DMN) (12 regions)"
                      ]
brain_networks_with_names = list(zip(brain_networks, brain_network_names))
copulas_resx = []
for brain_network, brain_network_name in brain_networks_with_names:
    T_bn = T_raw[:, :, brain_network]

    k = 2
    r = range(T_bn.shape[2])
    n = T_bn.shape[0]
    n_combs = len(list(combinations(r, k)))

    Tk_spearman = np.zeros((n, n_combs))

    for i, ts in tqdm(enumerate(T_bn), total=len(T_bn)):
        for j, simplex in enumerate(combinations(r, k)):
            z = spearmanr(ts[:, simplex])
            #print(z)
            Tk_spearman[i, j] = z.correlation
    # print((Tk_spearman != Tk_spearman2).sum())
    
    copulas_resx.append([logistic_score(Tk_spearman, y)])
    print(copulas_resx[-1], brain_network_name)

  0%|          | 0/956 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

['0.77470 ± 0.02940'] visual cortex (12 regions)


  0%|          | 0/956 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

['0.75407 ± 0.02701'] frontoparietal network (FPN) (10 regions)


  0%|          | 0/956 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

['0.77031 ± 0.02873'] default mode network (DMN) (12 regions)


In [81]:
tmp = np.array([[1, 2, 3, 4, 5], [5, 6, 7, 8, 7]]).T
print(spearmanr(tmp).correlation)
print(Copula(tmp).spearman())

0.8207826816681233
-2.0


In [80]:
class Copula:
    
    def __init__(self, XY):
        
        self.n, self.d = XY.shape
        if self.d<2:
            raise ValueError(f"Data should be at least two-dimensional, but d={self.d}.")
            
        # order statistics
        XY_order = np.sort(XY, axis=0)
        
        # set C
        self.C = np.zeros((self.n,self.n))

        for i in range(self.n):
            for j in range(self.n):
                num = np.sum(np.logical_and(XY[:,0] <= XY_order[i,0], XY[:,1] <= XY_order[j,1])) / self.n
                self.C[i,j] = num

        # set P
        i_n = np.arange(1, self.n+1) / self.n
        j_n = np.arange(1, self.n+1) / self.n
        self.P = np.multiply.outer(i_n, j_n)
        
    def spearman(self):
        
        if self.d!=2:
            raise ValueError(f"Spearman's rho is the bivariate measure, but d={self.d}.")
        
        #h_rho = 12/(self.n**2-1)
        return 3 * (4 / (self.n**2-1)* np.sum(self.C - self.P) - 1)
        #return h_rho * np.sum(self.C - self.P) # TODO: general d
    
    def schweizer_wolff(self, norm="inf"):
        
        if norm not in [1, 2, "inf"]:
            raise ValueError("Norm should be '1', '2', or 'inf'.")
            
        # normalizing factors
        h_1 = ((1/(self.d+1)) - (1/(2**self.d))) ** -1
        h_2 = 90 # TODO
        h_inf = ((1/self.d)**(1/(self.d-1)) * (1 - (1/self.d)))**-1
        
        if norm==1:
            delta = h_1 / (self.n**self.d-1) * np.sum(np.abs(self.C - self.P))
        elif norm==2:
            delta_sq = h_2 / (self.n**self.d-1) * np.sum((self.C - self.P)**2)
            delta = np.sqrt(delta_sq)
        elif norm=="inf":
            delta = h_inf * np.max(np.abs(self.C - self.P))
            
        return delta

**Other**

In [182]:
def genX2(X_bn):
    X2 = []
    for sigma in X_bn:
        comp = sigma.shape[0]
        Hs = [0] * comp
        Is = []
        for i in range(comp):
            Hs[i] = gaussian_H(sigma, [i])
            for j in range(i):
                Is.append(I(sigma, [j, i], precalc=Hs))
        X2.append(Is)
    X2 = np.array(X2)
    return X2

In [189]:
def genXpairs(X_bn):
    r = range(X_bn.shape[1])
    n_pairs = len(list(combinations(r, 2)))

    X_pairs = np.zeros((n, n_pairs))

    for i, S in enumerate(X_bn):
        for j, simplex in enumerate(combinations(r, 2)):
            simplex_idx = np.ix_(simplex, simplex)
            X_pairs[i,j] = mutual_information(S[simplex_idx])
    return X_pairs

In [194]:
def genX3(X_bn):
    X3 = []
    for sigma in X_bn:
        comp = sigma.shape[0]
        Hs = [0] * comp
        Is = []
        for i in range(comp):
            Hs[i] = gaussian_H(sigma, [i])
            for j in range(i):
                for k in range(j):
                    Is.append(I(sigma, [k, j, i], precalc=Hs))
        X3.append(Is)
    X3 = np.array(X3)
    return X3

In [121]:
logistic_score(X, y)

  0%|          | 0/10 [00:00<?, ?it/s]

0.74206 ± 0.03105


In [122]:
logistic_score(X_log, y)

  0%|          | 0/10 [00:00<?, ?it/s]

0.74865 ± 0.02958


In [123]:
def gaussian_H(sigma, inds):
    sigma = sigma[np.ix_(inds, inds)]
    det = np.linalg.det(sigma) if isinstance(sigma, np.ndarray) else sigma
    return (len(inds) * np.log(2 * np.pi * np.e) + np.log(det)) / 2

def I(sigma, inds, precalc=None):
    # not optimal if called for all inds combinations -- precalculate gaussian_H for all singletones
    res = -gaussian_H(sigma, inds)
    if precalc is None:
        for i in inds:
            res += gaussian_H(sigma, [i])
    else:
        for i in inds:
            res += precalc[i]
    return res

In [143]:
X2 = []
for sigma in X_dmn:
    comp = sigma.shape[0]
    Hs = [0] * comp
    Is = []
    for i in range(comp):
        Hs[i] = gaussian_H(sigma, [i])
        for j in range(i):
            Is.append(I(sigma, [j, i], precalc=Hs))
    X2.append(Is)
X2 = np.array(X2)
X2.shape

(956, 66)

In [153]:
X22 = X_dmn.copy()
for i, S in enumerate(X_dmn):
    for simplex in product(range(12), repeat=2):
        #print(simplex)
        if simplex[0] != simplex[1]:
            X22[(i, *simplex)] = I(S, simplex)
X2_full = corrmat2vec(X22)
X2_full_log = corrmat2vec(X22, take_logm=True)

In [132]:
logistic_score(X2, y)

  0%|          | 0/10 [00:00<?, ?it/s]

0.75116 ± 0.03073


In [154]:
logistic_score(X2_full, y)
logistic_score(X2_full_log, y)

  0%|          | 0/10 [00:00<?, ?it/s]

0.75127 ± 0.03087


  0%|          | 0/10 [00:00<?, ?it/s]

0.75524 ± 0.02657


In [238]:
X3 = []
for sigma in X:
    comp = sigma.shape[0]
    Hs = [0] * comp
    Is = []
    for i in range(comp):
        Hs[i] = gaussian_H(sigma, [i])
        for j in range(i):
            for k in range(j):
                Is.append(I(sigma, [k, j, i], precalc=Hs))
    X3.append(Is)
X3 = np.array(X3)
X3.shape

(564, 220)

In [81]:
n_pairs = len(list(combinations(dmn, 2)))
n = X.shape[0]

X_pairs = np.zeros((n, n_pairs))

for i, S in enumerate(X_dmn):
    for j, simplex in enumerate(combinations(np.arange(12), 2)):
        simplex_idx = np.ix_(simplex, simplex)
        X_pairs[i,j] = mutual_information(S[simplex_idx])

/tmp/ipykernel_4644/1739575051.py:13: RuntimeWarning: invalid value encountered in log
  return -0.5 * np.log(np.linalg.det(S))


In [83]:
logistic_score(X_pairs, y)

  0%|          | 0/10 [00:00<?, ?it/s]

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [291]:
print("\npairs\n", logistic_score(X2, y), sep='')
print("\ntriples\n", logistic_score(X3, y), sep='')
print("\nboth\n", logistic_score(np.hstack((X2, X3)), y), sep='')


pairs
Train: 1.0
Test: 1.0

triples
Train: 1.0
Test: 1.0

both
Train: 1.0
Test: 1.0


In [289]:
n_repeats, n_splits = 10, 5
accuracy = np.zeros((n_repeats, n_splits))
for i in tqdm(range(n_repeats)):

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=i)

    for j, (train_idx, test_idx) in enumerate(skf.split(X2, y)):

        X_train, X_test = X2[train_idx], X2[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        #print(y_train.shape, y_test.shape)

        scaler = StandardScaler()
        model = LogisticRegression(C=5, max_iter=500, random_state=i*n_repeats+j)
        clf = Pipeline([("scaler", scaler), ("model", model)]).fit(X_train, y_train)

        y_hat = clf.predict(X_test)
        accuracy[i, j] = clf.score(X_test, y_test)

        #accuracy[i,j] = accuracy_score(y_test, y_hat)
print(accuracy)
print(f"{accuracy.mean():.4f} ± {accuracy.std():.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
1.0000 ± 0.0000


---

In [23]:
X_test = []
y_test = []
for subj_id in SUBJ_IDS:
    for session_id in SESSION_IDS:
        for run_id in RUN_IDS:
            Sigma = np.corrcoef(ts2[(subj_id, session_id, run_id)])
            X_test.append(Sigma[np.triu_indices(Sigma.shape[0])])
            y_test.append(subj_id)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [24]:
clf.score(X_test, y_test)

0.6666666666666666

In [6]:
t1.shape

(200, 116)

In [ ]:
mri_file = './raw/timeseries/subj01/timeseries_session01_run01.nii.gz'
img = nib.load(mri_file)

In [ ]:
print(img.shape)
print(img.header.get_zooms())

Четвёртая размерность -- время.

In [13]:
mri_file = './raw/timeseries/subj01/timeseries_session02_run04.nii.gz'
img = nib.load(mri_file)
print(img.shape)
print(img.header.get_zooms())

(81, 104, 83, 226)
(1.8, 1.8, 1.8, 1.3333334)


In [11]:
mri_file = './raw/timeseries/subj05/timeseries_session01_run01.nii.gz'
img = nib.load(mri_file)
print(img.shape)
print(img.header.get_zooms())

(79, 97, 78, 226)
(1.8, 1.8, 1.8, 1.3333334)
